# A pipeline for recording, processing & successfully merging e-phys, IMU and tracking data (for further analyses).
### author: github/bartulem


#### **[Step 0]** Considerations before, during and after conducting the experiments.
1. Set up [SpikeGLX](https://billkarsh.github.io/SpikeGLX/) to accommodate your specific recording configuration.
2. Turn on and calibrate the IMU (system calibration does not need to be 3, but others do before the rat is plugged on).
3. Open Motive.
3a. Check whether the system is calibrated (continuous calibration should be on).
3b. Check whether the sync configuration is set to "Recording Gate".
3c. If necessary, for each camera change strobe light settings to continuous light.
3d. Check that the rigid bodies for the head & arena LEDs exist (this enables on-line automatic labeling).
3e. Check whether the acquisition directory is the correct one.
4. Put three circular markers on the back of the animal and plug it on.
5. Conduct the recording.
5a. If the data acquisition looks OK in SpikeGLX, start recording.
5b. Start recording in Motive.
5c. Start acquiring data on the IMU.
5d. Use the button LEDs to flash light many times (at least 100) during the recording.
5e. Stop acquiring data on the IMU.
5f. Stop recording in Motive.
5g. Stop recording in SpikeGLX.
6. It's good practice to label the back points immediately in Motive (the head & LEDs should be labeled already, if step 3d was implemented) and export the data in a .csv file.

#### **[Step 1]** Merge Neuropixel sessions to run Kilosort2. This step is *optional*, you can skip it if you are interested in only one session.
In the cell below, you determine the directories where the Neuropixel .bin files are (all associated files should be in the same directory), the desired name of the future merged/pkl files and their 
destination, the number of channels on the probe and whether to do the merging through the cmd prompt. Along with the merged file, this code outputs a .pkl file with information about changepoints 
of the merged sessions (necessary for extracting spike times later).

In [ ]:
from kisn_pylab import concatenateNPX

fileDirectories = [r'D:\SGL_DATA\test_concatenation']
newFileNames = [r'D:\SGL_DATA\test_concatenation\05022020_both_distal_g0_t0.imec0.ap.bin']
futurepkls = [r'D:\SGL_DATA\test_concatenation\05022020_both_distal_g0_t0.imec0.ap.pkl']
cmdPrompt = 1
nchan = 385

In [ ]:
for file_dir, new_file_name, future_pkl in zip(fileDirectories, newFileNames, futurepkls): 
    concatClass = concatenateNPX.Concat(file_dir, new_file_name, future_pkl)
    concatClass.concat_npx(cmd_prompt=cmdPrompt, nchan=nchan)

#### **[Step 2]** Run Kilosort2 through Python.
This assumes you are happy with *everything* in the config file. If you need to modify anything, either the code needs to change or you complete this step in Matlab.
If this doesn't bother you, then you should do the following:
0. Download/clone [Kilosort2](https://github.com/MouseLand/Kilosort2) and set up the config, master and CUDA files accordingly.
1. Install [matlab engine](https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html) (as an admin!).
2. Kilosort2 runs on all the .bin files in the given directory below. Make sure that this is what you want.
3. Don't use my Kilosort2 directory, but rather your own (created in step 0).
4. Set the file and Kilosort2 directories, and run the cell below.
NB: While Kilosort2 is running, it's a good opportunity to label the tracking data if you haven't done so already.

In [ ]:
from kisn_pylab import runKilosort

file_dir = r'D:\SGL_DATA\test_concatenation'
kilosort2_dir = r'A:\group\bartulm\Kilosort2-master'

In [ ]:
runKilosort.run_kilosort(file_dir, kilosort2_dir)

#### **[Step 3]** Arbitrate what is noise and what are clusters in Phy.
1. Install Phy: [Phy v2.0](https://github.com/cortex-lab/phy)
2. Navigate to the directory where Kilosort2 results were saved, open powershell and type "cmd", followed by "activate phy2", followed by "phy template-gui params.py".
3. Complete the manual curation ([Phy tutorial](https://phy.readthedocs.io/en/latest/)) and save your work.

#### **[Step 4]** Read in the sync events (make sure the PC has enough memory to run this, say 64Gb RAM) and put them in separate .txt files.
If you haven't done so already, label the tracked rigid bodies and marker sets in Motive (read the tutorial if you need) and export the data:
1. File > Export Tracking Data.
2. Put OFF in the following variables: (1) Unlabeled markers, (2) Rigid Bodies, (3) Rigid Body markers, (4) Bones, (5) Bone markers.
3. Click "Export" and you should have created a .csv file.

You set the list with all the files whose sync events you'd like to read, and likewise the path of the future sync .pkl file and the future IMU data .pkl file.
There are three additional parameters: nchan (the total number of channels on the probe, same as above), sync_chan (the specific sync port channel on the probe)
and ledoff (consider led off events in the analyses).

In [ ]:
from kisn_pylab import readSyncEvents

npx_files = [r'D:\SGL_DATA\05022020_session2_distal_g0\05022020_session2_distal_g0_t0.imec0.ap.bin']
track_file = r'A:\store\Bartul\neuropixel\distal05022020\tracking\Take 2020-02-05 04.01.13 PM (2).csv'
imu_file = r'A:\store\Bartul\neuropixel\scripts\CoolTerm Capture 2020-02-12 14-43-06.txt'
sync_df = r'A:\store\Bartul\neuropixel\sync_df_05022020_distal_session2.pkl'
imu_pkl = r'A:\store\Bartul\neuropixel\scripts\CoolTerm Capture 2020-02-12 14-43-06.pkl'
nchan = 385
sync_chan = 385
ledoff = False

In [ ]:
readClass = readSyncEvents.Sync(npx_files, sync_df)
readClass.read_se(nchan=nchan, sync_chan=sync_chan, ledoff=ledoff, track_file=track_file, imu_file=imu_file, imu_pkl=imu_pkl)

#### **[Step 5]** Load the sync data from the .pkl file(s) and analyze how well the tracking/IMU data are synced with NPX data.
Before running this step, if you want to plot - make sure you have [plotly](https://plotly.com/python/getting-started/?utm_source=mailchimp-jan-2015&utm_medium=email&utm_campaign=generalemail-jan2015&utm_term=bubble-chart) installed.
You set the paths to the the sync_df .pkl files are. There are two additional parameters: npx_sampling_rate (defaults to 3e4) and to_plot (whether to plot the cross-validation results).

In [ ]:
from kisn_pylab import syncCheck

pkl_dirs = [r'A:\store\Bartul\neuropixel\sync_df_05022020_distal_session2.pkl']
to_plot = 1

In [ ]:
syncClass = syncCheck.Sync(pkl_dirs)
syncClass.estimate_sync_quality(to_plot=to_plot)

#### **[Step 6]** Split clusters back into individual sessions and get spike times. This step should be completed also if you only spike sorted one session.
Note that you set the number of channels on the probe, whether you have only one session or not, and the minimum number of spikes in one session to consider the cluster worthy of saving.

In [ ]:
from kisn_pylab import saveSpikeTimes

thedir = r'A:\store\Bartul\neuropixel\distal05022020'
pklfile = '05022020_both_distal_g0_t0.imec0.ap.pkl'
nchan = 385
onesession = 1
minspikes = 100

In [ ]:
sstClass = saveSpikeTimes.sst(thedir)
sstClass.splitClusters(onesession=onesession, minspikes=minspikes, nchan=nchan, pklfile=pklfile)

#### **[Step 7]** Create .pkl file for GUI.
You set the paths where the .csv (tracking), .pkl (sync event) files are. If necessary, you can add the following variables: (1) frame_rate (for tracking; such that it's not read from the .pkl file),
(2) npx_sampling_rate (default is 3e4), (3) reference_probe (default is 0 for imec0) and pass them to the method.

In [ ]:
from kisn_pylab import motive2GUI

thecsvs = [r'A:\store\Bartul\neuropixel\distal05022020\tracking\Take 2020-02-05 04.01.13 PM (2).csv']
thepkls = [r'A:\store\Bartul\neuropixel\05022020_distal_session2.pkl']

In [ ]:
for the_csv, the_pkl in zip(thecsvs, thepkls):
    mtgClass = motive2GUI.MotiveGUI(the_csv, the_pkl)
    mtgClass.csv_to_pkl()

#### **[Step 8]** Create the head in the GUI (trackedpointdata_V3_5_LEDs.py version), load the spiking .mat files and export everything as a .mat file. Now you are ready to analyze!